---
title: "Sentinel-1"
aliases: 
  - /Data/Sentinel1.html
  - /Data/Sentinel1.qmd
format:
  html:
    code-fold: true
execute:
  echo: false
  output: asis
jupyter: python3
image: https://ai4edatasetspublicassets.blob.core.windows.net/assets/pc_thumbnails/sentinel-1-grd.png
---

In [ ]:
constellation = "Sentinel-1"

The [Sentinel-1 radar imaging mission](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1){target="_blank"} is composed of a constellation of two polar-orbiting satellites providing continuous all-weather, day and night imagery for Land and Maritime Monitoring. C-band synthetic aperture radar imaging has the advantage of operating at wavelengths that are not obstructed by clouds or lack of illumination and therefore can acquire data during day or night under all weather conditions.

**The end of mission of the Sentinel-1B satellite has been declared in July 2022**
<br>On 23 December 2021, Copernicus Sentinel-1B experienced an anomaly related to the instrument electronics power supply provided by the satellite platform, leaving it unable to deliver radar data. Despite all investigations and recovery attempts, ESA and the European Commission had to announce that it is the end of the mission for Sentinel-1B. Copernicus Sentinel-1A remains fully operational. More information about the end of the mission for the Sentinel-1B satellite can be found on the webpage [Mission ends for Copernicus Sentinel-1B satellite](https://www.esa.int/Applications/Observing_the_Earth/Copernicus/Sentinel-1/Mission_ends_for_Copernicus_Sentinel-1B_satellite){target="_blank"}.
<br>In response to the loss of Sentinel-1B, **the mission observation scenario of Sentinel-1A was adjusted**, affecting the nominal global coverage frequency. An up-to-date overview of the observation scenario in place can be consulted on the webpage [Sentinel-1 Observation Scenario](https://sentinel.esa.int/web/sentinel/missions/sentinel-1/observation-scenario){target="_blank"}. Some regions are currently not observed by Sentinel-1. Nevertheless, the regions that are still observed, now have a repeat cycle of 12 days under a one-satellite constellation scenario, which affects possible interferometric analyses.


Sentinel data products are made available systematically and free of charge to all data users including the general public, scientific and commercial users. These [data products](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel-1/data-products){target="_blank"} are available in single polarisation for Wave mode and dual polarisation or single polarisation for SM, IW and EW modes.


In [ ]:
import json
from IPython.display import display, Markdown, Latex,HTML
from tabulate import tabulate

import sys
sys.path.append('..')
from Datafunctions import data_availability, filter_levels

In [ ]:
#load the json metadata file

meta = None
with open("../collections.json") as f:
    meta = json.load(f)["collections"]

meta = [ c for c in meta if constellation in c["constellation"] ]

In [ ]:
# function to fetch thumbnail url

def get_thumbnail():
    thumb_url = c.get("assets", {}).get("thumbnail",{}).get("href", "")
    return thumb_url

In [ ]:
#function to fetch the stac url for the dataset
def get_stac():
    try:
        stac_extension = c["stac_extensions"]
        link = None  # initialize link variable to None

        for item in stac_extension:
            if item.startswith('http'):
                stac_url = f"""STAC: <a href="{item}" target="_blank">{item}</a>"""
                break  # exit the loop once you've found the link        
    except Exception:
        stac_url = ""
    return stac_url

In [ ]:
# function to search for finding links within the metadata based on specific keyword

def find_link(rel=None,href_contains=None, title_contains=None):
    results = [ l["href"] for l in c.get("links",[]) if (rel is None or l["rel"]==rel) and (href_contains==None or href_contains in l["href"]) and (title_contains==None or title_contains in l["title"]) ]
    if len(results) == 0:
        return None
    else:
        return results[0]

In [ ]:
#get link for wmts
def get_wmts():
    url = ""
    try:
        wmts = find_link(rel="wmts",title_contains="Web Map Tile Service")
        if wmts is not None:
            url = f"""WMTS:  <a href="{wmts}" target="_blank">{wmts}</a>""" 
    except Exception:
        url = ""
    return url

In [ ]:
#get link for source
def get_source():
    url = ""
    try:
        source = find_link(rel="source")
        if source is not None:
            url = f"""Source:  <a href="{source}" target="_blank">{source}</a>""" 
    except Exception:
        url = ""
    return url

In [ ]:
#get link for more information of the collection
def get_moreinfo():
    url = ""
    try:
        info = find_link(rel="MoreInformation")
        if info is not None:
            url = f"""More Information:  <a href="{info}" target="_blank">{info}</a>""" 
    except:
        url = ""
    return url

In [ ]:
#function to combine all the fetched links under the detailed block

def necessary_links():

    # list of all links
    links = [get_stac(), get_wmts(), get_source(), get_moreinfo()]

    # join non-empty links using HTML list tags
    non_empty_links = [link for link in links if link]
    if non_empty_links:
        link_items = [f"<li>{link}</li>" for link in non_empty_links]
        link_block = f"""<h5>Useful Links</h5><ul>{"".join(link_items)}</ul>"""
    else:
        link_block = ""

    return link_block

In [ ]:
# function enclosing thumbnail block 

def thumbblock(src,userguide, opensearch, odata, gsd,revisit_time, datatype, frequency):

    if src or userguide or opensearch or odata or gsd or revisit_time or datatype or frequency:
        if src:
            thumb =  f"""<div class="row" style="display: flex; justify-content: space-around"><div class="column"><div class="badge-row">{userguide}{opensearch}{odata}</div><div class="badge-row gutter">{gsd}{revisit_time}{frequency}</div><div class="badge-row">{datatype}</div></div><div class="column" style="margin-right:10%; margin-left:auto; text-align: right;"><figure><img width="165" height="150" src={src}><figcaption align="center">{databrowser}</figcaption></figure></div></div>"""
        else:
            thumb =  f"""<div class="row" style="display: flex; justify-content: space-around"><div class="column"><div class="badge-row">{userguide}{opensearch}{odata}</div><div class="badge-row gutter">{gsd}{revisit_time}{frequency}</div><div class="badge-row">{datatype}</div></div><div class="column" style="margin-right:10%; margin-left:auto; text-align: right;"></div></div>"""
        return thumb
    else:
        return ''

In [ ]:
# get extension detail
def get_extent():
    try:
        extent = c["extent"]
        spatial = extent['spatial']['bbox']
        spatial = f'<h5>Spatial Extent</h5> {spatial}'
    except Exception:
        spatial = ""
    try:
        temporal = extent['temporal']['interval']
        temporal = f'<h5>Temporal Interval</h5> {temporal}'
    except Exception:
        temporal = ""
    return spatial,temporal

In [ ]:
def get_bandtable():
    tabletitle = "Spectral Bands"
    try:
        band_num = len(c['summaries']['eo:bands'])
        t = []
        headers = ["Band Name", "Common Name", "GSD(m)", "Center Wavelength(μm)"]
        empty_columns = [True] * len(headers)  # Track empty columns
        
        for i in range(band_num):
            band_data = c['summaries']['eo:bands'][i]
            band = band_data.get("name", "")    
            c_name = band_data.get("common_name", "")  
            b_gsd = band_data.get("gsd", "")
            b_wavelength = band_data.get("center_wavelength", "")               
            row = [band, c_name, b_gsd, b_wavelength]
            t.append(row)
            
            # Check for empty columns
            for j, value in enumerate(row):
                if value:
                    empty_columns[j] = False
        
        # Remove empty columns from headers and table rows
        headers = [header for i, header in enumerate(headers) if not empty_columns[i]]
        t = [[row[i] for i, value in enumerate(row) if not empty_columns[i]] for row in t]
        
        # Check if all columns are empty
        if len(headers) == 0:
            table = ""
        else:
            table = tabulate(t, headers=headers, tablefmt='html', floatfmt=".4f", stralign="center", numalign="center")
            # Set the minimum width of each column to 100 pixels
            table = table.replace("<table>", '<table class="table">')
            table = f"""<h5>{tabletitle}</h5>{table}"""
    except:
        table = ""
    
    return table


In [ ]:
#function encloding detail block (block under **Further details about the collection**)

def detailblock(ID,license_button,cite):
    #get temporal and spatial extent
    spatial, temporal = get_extent()
    
    # include spectral band table
    table = get_bandtable()

    # get necessary links
    link_block = necessary_links()

    #combine all the details to be shown within the detailed block
    if any(val for val in (table, link_block) if not val.isspace() and val != ""):
        bandinfo = f"""<button onclick="toggleContent('{ID}')" class="expand-collapse">Further details about the data collection</button><div class="expand-content" id="{ID}" style="display:none;"><div class="row" style="display: flex; justify-content: space-around"><div class="column">{license_button}&nbsp;&nbsp;{cite}</div></div><div class="row" style="display: flex; justify-content: space-around"><div class="column">{spatial}</div><div class="column">{temporal}</div></div><div>{table}</div><div>{link_block}</div></div>"""
    else:
        bandinfo = ""
    return bandinfo

In [ ]:
# Display the baseline details of the collections
def baselineinfo(filename): 
    # Read the content of the Markdown file
    with open(filename, 'r') as file:
        sentinel2_baseline = file.read()
    info = sentinel2_baseline
    return info

In [ ]:
# function to toggle detail block

def togglescript():
    script = f"""<script>function toggleContent(id){{var x = document.getElementById(id);if (x.style.display === "none"){{x.style.display = "block";}} else {{x.style.display = "none";}}}}</script>"""
    return script

In [ ]:
# for loop to display the content
text = ''
categories=[]

for c in meta:

    additional_info = ""
    title = c["title"]
    resolution = c.get("gsd", "")
    revisit = c.get("revisit", "")
    datatype = c.get("type", "")   
    collectionID = c.get("id", "")       

    try:
        for i in c["data_product"]:
            levelID = c["data_product"][i]["Level_ID"]
            about_product = c["data_product"][i]["description"]
            levelids.append((levelID,about_product))
    except Exception:
        levelID = ""
        about_product = ""

    license_type = c.get("license", "")
    sci_cite = c.get("sci:citation", "")
    frequency = c.get('summaries',{}).get('Frequency',"")
    category = c.get("Category", "")
    if category != "":
        categories.append(category)
    else:
        pass

    if collectionID == "SENTINEL2_L2A":
        additional_info = baselineinfo("../Others/Sentinel2_L2A_baseline.qmd")
    if collectionID == "SENTINEL2_L1C":
        additional_info = baselineinfo("../Others/Sentinel2_L1C_baseline.qmd")
    if collectionID == "SENTINEL2_L3":
        additional_info = baselineinfo("../Others/Sentinel2_Mosaic_Algorithm.qmd")
    if collectionID == "SENTINEL1_L3":
        additional_info = baselineinfo("../Others/Sentinel1_Mosaic_Algorithm.qmd")
    else:
        pass


    # include offer table
    offer = data_availability.main(c)
    
    links = { l["rel"] + l.get("title",""):l["href"] for l in c.get("links",[])}
    userguide = find_link(rel="about",title_contains="User")
    opensearch = find_link(rel="opensearch")
    odata = find_link(rel="odata")
    databrowser = find_link(rel="browser",title_contains="Data-Browser")
    license_url = find_link(rel="license")
    cite = find_link(rel="cite-as")
    
    # create a button for userguide

    if userguide is not None:
        userguide = f"""[![User guide](https://img.shields.io/badge/-User_guide-77cc09)]({userguide}){{target='_blank'}}"""
    else:
        userguide = ""

    # create a button for cite
    if cite is not None:
        cite = f"""[!["Cite"](https://img.shields.io/badge/-Cite-77cc09)]({cite}){{target='_blank'}}"""
    else:
        cite = ""

    # create a button for license 
    if sci_cite is not None:
        if license_url is not None and  license_url != "":
            license_button = f"""<a href="{license_url}" target="_blank">{sci_cite}</a>"""
        else:
            license_button = f"""{sci_cite}"""
    else:
        license_button = ""

    # create a button to the browser 
    if databrowser is not None:
        databrowser = f"""<a class="clickable" href="{databrowser}" target="_blank">View in browser</a>"""
    else:
        databrowser = ""

    # create a button showing the resolution of the collection
    if resolution is not None and  resolution != "":
        gsd = f"""<img src="https://img.shields.io/badge/resolution-{resolution}m-0A4393" />"""
    else:
        gsd = ""

    # create a button showing the revisit time of the collection
    if revisit is not None and  revisit != "":
        revisit_time = f"""<img src="https://img.shields.io/badge/revisit-{revisit}--day-0A4393"/>"""
    else:
        revisit_time = ""

    # create a button showing the frquency of the product
    if frequency is not None and  frequency != "":
        Updated_freq = f"""<img src="https://img.shields.io/badge/Update_Frequency-{frequency}-0A4393"/>"""
    else:
        Updated_freq = ""

    # create a button specifying if it is Analysis ready data or unprojected
    if "ARD" in datatype:
        datatype = f"""![](https://img.shields.io/badge/{datatype}-77cc09) """
    elif "Unprojected" in datatype:
        datatype = f"""![](https://img.shields.io/badge/{datatype}-important) """
    else:
        datatype = ""

    # create a button to api url
    if opensearch is not None:
        opensearch = f"""[![Catalog API:OpenSearch](https://img.shields.io/badge/-Catalog_API:OpenSearch-77cc09?style=flat)
]({opensearch}){{target='_blank'}}"""
    else:
        opensearch = ""
    
    if odata is not None:
        odata = f"""[![Catalog API:OData](https://img.shields.io/badge/-Catalog_API:OData-77cc09?style=flat)
]({odata}){{target='_blank'}}"""
    else:
        odata = ""

    text = text + f"""

<div class="{category}" style="display: block;">

<div>

## {title}

{thumbblock(get_thumbnail(),userguide,opensearch,odata,gsd,revisit_time,datatype,Updated_freq)}

#### Overview

{c["description"]}

{offer}

{detailblock(collectionID,license_button,cite)}

{additional_info}

</div>

</div>

"""

In [ ]:
text = filter_levels.createcheckboxes(sorted(set(categories))) +text + togglescript()

display(Markdown(text))

# Derived products & Processing options

Sentinel-1 data can be accessed and processed in different ways within the Copernicus Data Space Ecosystem. Below we have compiled an overview of all the options to help you decide which one to use.

## [Sentinel-1 RTC](https://documentation.dataspace.copernicus.eu/Data/Additional.html#sentinel-1-rtc) 
[Sentinel-1 RTC](https://documentation.dataspace.copernicus.eu/Data/Additional.html#sentinel-1-rtc) (Radiometric Terrain Correction) SAR Backscatter is a product processed from Sentinel-1 GRD data and compliant with [CEOS Analysis Ready Data for Land (CARD4L) specifications](https://ceos-dev.ceos.org/ard/){target="_blank"} for Normalised Radar Backscatter (NRB) products. Orthorectification is based on Copernicus DEM and no speckle filtering is applied. ([Additional product information](https://sentinels.copernicus.eu/web/sentinel/sentinel-1-ard-normalised-radar-backscatter-nrb-product){target="_blank"})

## [Sentinel Hub processing options](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-options) 

Sentinel Hub offers the following processing options in the [Sentinel-1 GRD processing chain](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-chain):

- [Backscatter coefficients](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-options):

    - **beta0 (ellipsoid)**
    - **sigma0 (ellipsoid)**
    - **gamma0 (ellipsoid)**
    - **gamma0 (terrain)** &rarr; this gamma0 RTC option can only be performed if *orthorectification* is enabled

- [Lee Speckle Filtering](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-options) applied on source data after calibration and noise removal
- [Radiometric Terrain Correction (RTC)](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-options) can be enabled by setting the backscatter coefficient to *gamma0 (terrain)* and enabling *orthorectification*
- [Orthorectification](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Data/S1GRD.html#processing-options) with Range-Doppler terrain correction using one of the following DEMs:

    - **Copernicus 10m/30m DEM** (10m resolution inside [39 European states](https://spacedata.copernicus.eu/collections/copernicus-digital-elevation-model){target="_blank"} including islands and 30m elsewhere.)
    - **Copernicus 30m DEM**
    - **Copernicus 90m DEM**

## openEO processing options 

When working with the SENTINEL1_GRD data collection through openEO, SAR backscatter computation is automatically applied. Unfortunately, the default backscatter coefficient "gamma0-terrain" is not yet supported in the openEO backend implementation of Copernicus Data Space Ecosystem, typically resulting in an error like "Backscatter coefficient 'gamma0-terrain' is not supported."

As a workaround, it is currently recommended to explicitly specify the [`sar_backscatter()`](https://processes.openeo.org/draft/#sar_backscatter){target="_blank"} process with the supported coefficient "sigma0-ellipsoid".

- **sigma0-ellipsoid**: ground area computed with ellipsoid earth model

For example:

```python

sentinel1 = connection.load_collection(
    "SENTINEL1_GRD",
    temporal_extent = ["2022-06-04", "2022-08-04"],
    bands = ["VV","VH"]
)

  sentinel1 = sentinel1.sar_backscatter(
      coefficient='sigma0-ellipsoid')
```

The product is orthorectified using the Copernicus 30m DEM. No RTC or speckle filtering is applied to this product.


## [On-demand processing options](https://documentation.dataspace.copernicus.eu/APIs/On-Demand%20Production%20API.html) 

Processing of CARD-BS and COH6/COH12 products can be requested [on demand](https://documentation.dataspace.copernicus.eu/APIs/On-Demand%20Production%20API.html): 

- [Sentinel-1 (CARD-BS) BackScatter](https://documentation.dataspace.copernicus.eu/APIs/On-Demand%20Production%20API.html)

    - This processing option contains gamma0 geometric terrain correction (orthorectification) using Copernicus 30m DEM (identical to the *gamma0 (ellipsoid)* backscatter coefficient with enabled *orthorectification* option in Sentinel Hub processing options.) No RTC or speckle filtering is applied to this product.
    - [Additional information](https://creodias.eu/eodata/sentinel-1/sentinel-1-l2-backscatter-bs/){target="_blank"}

- [Sentinel-1 (CARD-COH) Coherence](https://documentation.dataspace.copernicus.eu/APIs/On-Demand%20Production%20API.html)

    - The Sentinel-1 CARD COH (Copernicus Analysis Ready Data Coherence) processor generates a Sentinel-1 Level 2 product describing the coherence of a pair of images - 12 days apart. The product is orthorectified using Copernicus 30m DEM but no RTC or speckle filtering is applied.
    - [Additional information](https://creodias.eu/eodata/sentinel-1/sentinel-1-l3-bs-monthly-com/){target="_blank"}.